# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
f64,str,str,str,str,str,bool
3.9172e17,"""409976367371330704""","""../../../../data/SampleHDS.jso…","""16140487010276616868""","""7602958664075868333""","""7103865483712133782""",false
1.2175e19,"""11257802366543574485""","""../../../../data/SampleHDS.jso…","""12200476650473293865""","""7602958664075868333""","""2037530117245275542""",true
1.6241e19,"""6624936581383903775""","""../../../../data/SampleHDS.jso…","""4996001417216770133""","""5137906438115743641""","""7103865483712133782""",false
3.3591e18,"""11257802366543574485""","""../../../../data/SampleHDS.jso…","""5668877670830418731""","""12848241752633552706""","""7103865483712133782""",false
1.4307e19,"""409976367371330704""","""../../../../data/SampleHDS.jso…","""6751794937911424182""","""12848241752633552706""","""260736542130782807""",true
2.4596e18,"""6624936581383903775""","""../../../../data/SampleHDS.jso…","""1226055995641522485""","""4518339848689972113""",null,false
7.7115e16,"""6624936581383903775""","""../../../../data/SampleHDS.jso…","""11585398652325949317""","""12848241752633552706""","""260736542130782807""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_CLV': 'PREDICTOR_0',
 'Customer_MaritalStatus': 'PREDICTOR_1',
 'filename': 'filename',
 'Customer_City': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
1.3657e18,"""12732749183414739901""","""../../../../data/SampleHDS.jso…","""4752058102665284145""","""FirstMortgage30yr""","""4487901229594076709""","""Rejected"""
1.1891e19,"""8866041553733105550""","""../../../../data/SampleHDS.jso…","""2291194945724959657""","""FirstMortgage30yr""","""17021559744653070236""","""Accepted"""
9.5426e17,"""17229910470229083885""","""../../../../data/SampleHDS.jso…","""3811096329090661384""","""MoneyMarketSavingsAccount""","""4487901229594076709""","""Rejected"""
8.3649e18,"""8866041553733105550""","""../../../../data/SampleHDS.jso…","""3241554837754797857""","""BasicChecking""","""4487901229594076709""","""Rejected"""
1.4053e19,"""12732749183414739901""","""../../../../data/SampleHDS.jso…","""10650316272041535014""","""BasicChecking""","""18172795986709107798""","""Accepted"""
1.2769e19,"""17229910470229083885""","""../../../../data/SampleHDS.jso…","""11963594887695444746""","""UPlusFinPersonal""",null,"""Rejected"""
4.3752e18,"""17229910470229083885""","""../../../../data/SampleHDS.jso…","""4519237799014449750""","""BasicChecking""","""18172795986709107798""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
5.6312e18,"""12965950700854461028""","""../../../../data/SampleHDS.jso…","""10019368320494160317""","""FirstMortgage30yr""","""9938380867965606470""","""Rejected"""
1.5045e19,"""8805169364686869144""","""../../../../data/SampleHDS.jso…","""8338853481069487416""","""FirstMortgage30yr""","""15787872950428691312""","""Accepted"""
8.4990e17,"""4792610448778148838""","""../../../../data/SampleHDS.jso…","""16571252424209782510""","""MoneyMarketSavingsAccount""","""9938380867965606470""","""Rejected"""
1.2769e18,"""8805169364686869144""","""../../../../data/SampleHDS.jso…","""6261726470952290473""","""BasicChecking""","""9938380867965606470""","""Rejected"""
1.6616e19,"""12965950700854461028""","""../../../../data/SampleHDS.jso…","""13670519945947685645""","""BasicChecking""","""2016705285168736516""","""Accepted"""
8.3926e17,"""4792610448778148838""","""../../../../data/SampleHDS.jso…","""17295786374092709127""","""UPlusFinPersonal""",null,"""Rejected"""
1.7525e19,"""4792610448778148838""","""../../../../data/SampleHDS.jso…","""14813960858206363483""","""BasicChecking""","""2016705285168736516""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_CLV=PREDICTOR_0
Customer_MaritalStatus=PREDICTOR_1
filename=filename
Customer_City=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
1.4097e19,"""8953836104091885106""","""5743676217813340133""","""FirstMortgage30yr""","""888337361043737642""","""Rejected"""
1.2029e19,"""4757318365571124941""","""17241651010184176855""","""FirstMortgage30yr""","""16248770842375249555""","""Accepted"""
5.6815e18,"""17947652281120269666""","""18265537837906333739""","""MoneyMarketSavingsAccount""","""888337361043737642""","""Rejected"""
1.1826e19,"""4757318365571124941""","""9072649960753862101""","""BasicChecking""","""888337361043737642""","""Rejected"""
7.8386e18,"""8953836104091885106""","""2500679204155009638""","""BasicChecking""","""2993335377860571078""","""Accepted"""
9.0594e18,"""17947652281120269666""","""18081424439623771463""","""UPlusFinPersonal""",null,"""Rejected"""
1.1655e19,"""17947652281120269666""","""2273871731013003625""","""BasicChecking""","""2993335377860571078""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'